# Save SDF for Compounds in the benchmark datasets

In [ ]:
import pandas as pd
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem
import os

activity_root = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/'
task_dict = {0: ('CHEMBL202', 'pIC50'), 1: ('CHEMBL235', 'pEC50'), 2: ('CHEMBL279', 'pIC50'), 3: ('CHEMBL2971', 'pIC50'), 
             4: ('CHEMBL333', 'pIC50'), 5: ('CHEMBL3820', 'pEC50'), 6: ('CHEMBL3976', 'pIC50'), 7: ('CHEMBL4422', 'pEC50')}
set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']

def save_sdf(activity_path, save_path):
    # 读取CSV文件
    df = pd.read_csv(activity_path)

    # 获取分子SMILES列的数据
    smiles_list = df['SMILES'].tolist()
    id_list = df['ChEMBL_Compound_ID'].tolist()

    # 计算并保存SDF文件
    for index, smiles in zip(id_list, smiles_list):
        output_file = save_path + f'{index}.sdf'
        if os.path.exists(output_file):
            continue
        try:
            mol = Chem.MolFromSmiles(smiles)
            hmol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(hmol,AllChem.ETKDG())
            print(AllChem.UFFOptimizeMolecule(hmol,1000))
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            writer = Chem.SDWriter(output_file)
            hmol.SetProp("_SMILES","%s"%smiles)
            writer.write(hmol)
            writer.close()
    #         print(f'Successfully save SDF for the compound: {index}')
        except:
            print(f'Fail to save SDF for the compound: {index} in the {set_name} set')

for key, value in task_dict.items():
    target, assay = value
    for set_name in tqdm(set_list):
        activity_path =f'{activity_root}/{target}/{assay}/{target}_{assay}_{set_name}.csv'
        save_path = f'{activity_root}/{target}/ligand/sdf/{set_name}/'
        save_sdf(activity_path, save_path)

# Prepare ligands in pdbqt format for molecular docking

In [ ]:
import os
from tqdm import tqdm
from rdkit import Chem

data_root = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/'
task_dict = {0: ('CHEMBL202', 'pIC50'), 1: ('CHEMBL235', 'pEC50'), 2: ('CHEMBL279', 'pIC50'), 3: ('CHEMBL2971', 'pIC50'), 
             4: ('CHEMBL333', 'pIC50'), 5: ('CHEMBL3820', 'pEC50'), 6: ('CHEMBL3976', 'pIC50'), 7: ('CHEMBL4422', 'pEC50')}
set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']

for key, value in task_dict.items():
    target, assay = value
    # List of input SDF files
    input_dir = f'{data_root}/{target}/ligand/sdf/'
    output_dir_addH = input_dir
    output_dir_pdbqt = f'{data_root}/{target}/ligand/pdbqt/'

    # Loop through input files
    for set_name in tqdm(set_list):
        input_files = os.listdir(input_dir + set_name)
        for input_file in tqdm(input_files):
            input_file_path = input_dir + set_name + '/' + input_file
            # Load the molecule from the SDF file
    #         mol = Chem.SDMolSupplier(input_file_path)[0]

    #         # Add explicit hydrogens
    #         mol = Chem.AddHs(mol)

    #         # Save the modified molecule with explicit hydrogens
    #         sdf_addH_save_path = output_dir_addH + set_name + '/'
    #         if not os.path.exists(sdf_addH_save_path):
    #                 os.makedirs(sdf_addH_save_path)
    #         Chem.SDWriter(sdf_addH_save_path + input_file).write(mol)

            # Construct the command
            pdbqt_save_path = output_dir_pdbqt + set_name + '/'
            if not os.path.exists(pdbqt_save_path):
                    os.makedirs(pdbqt_save_path)
            output_path = pdbqt_save_path + input_file[:-4] + '.pdbqt'
            if not os.path.exists(output_path):
                command = f"mk_prepare_ligand.py -i {input_file_path} -o {output_path}"
                # Execute the command
                !{command}

# Do AutoDock Vina on a batch of ligands and one protein

In [2]:
import subprocess
import pandas as pd
from tqdm import tqdm
import os

protein_name, pdb_name, pocket_num = 'CHEMBL3976', '4ebb', 2
# 定义输入输出文件路径列表
input_protein = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/protein/{pdb_name}.pdbqt'
input_dir_ligand = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/ligand/pdbqt/'
input_config_dir = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/protein/'
output_dir = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/{pdb_name}/vina/'
input_sets = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
command_template = "vina --receptor {} --ligand {} --config {} --exhaustiveness 32 --out {}"
# 循环处理每个文件路径
for i in range(pocket_num):
    input_config = input_config_dir + f'pocket_{i+1}.txt'
    for set_name in tqdm(input_sets):
        if not os.path.exists(output_dir + set_name):
            os.makedirs(output_dir + set_name)
        input_ligands = os.listdir(input_dir_ligand + set_name)
        for input_ligand in tqdm(input_ligands):
            # 提取文件名（不包含扩展名）
            filename = input_ligand.split('.')[0]
            output_path = output_dir + set_name + '/' + filename + f'_Pocket_{i+1}.pdbqt'
            if os.path.exists(output_path + '.txt'):
                os.rename(output_path + '.txt', output_path.replace('pdbqt', 'txt'))
            if os.path.exists(output_path):
                continue
            # 构建命令
            command = command_template.format(input_protein, input_dir_ligand + set_name + '/' + input_ligand, input_config, 
                                              output_path)
            # 执行命令并捕获输出结果
            output = subprocess.run(command, shell=True, capture_output=True, text=True)

            # 将输出结果保存到文件
            with open(output_path.replace('pdbqt', 'txt'), 'w') as f:
                f.write(output.stdout)


 56%|██████████████████████▌                 | 154/273 [00:01<00:00, 119.68it/s]


100%|█████████████████████████████████████████████| 6/6 [00:07<00:00,  1.31s/it]


# Save Vina scores into the datasets

In [3]:
import os, re
from tqdm import tqdm
from rdkit import Chem
import pandas as pd

task_dict = {'I1': ('CHEMBL202', 'pIC50', '1boz', 7, 1), 'E3': ('CHEMBL235', 'pEC50', '1zgy', 4, 2), 
             'I5': ('CHEMBL279', 'pIC50', '1ywn',3, 3), 'I4': ('CHEMBL2971', 'pIC50', '3ugc', 3, 3), 
             'I3': ('CHEMBL333', 'pIC50', '1ck7', 6, 2), 'E1': ('CHEMBL3820', 'pEC50', '3f9m', 6 ,1), 
             'I2': ('CHEMBL3976', 'pIC50', '4ebb', 2, 4), 'E2': ('CHEMBL4422', 'pEC50', '5tzr', 3, 3)}
protein_name, assay_type, pdb_name, pocket_num, pose_num = task_dict['I5']
# List of input SDF files
result_root = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}'
result_dir = f'{result_root}/{pdb_name}/vina/'
set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
csv_dir = f'{result_root}/{assay_type}/{protein_name}_{assay_type}_'
output_path = f'{result_dir}/{protein_name}_{assay_type}_'
    
# 定义正则表达式模式
pattern = r'^\s+(\d)\s+(-?\d+\.\d+)'
pattern_ = r'^\s+(\d)\s+(-?\d+)'
# new_column_names = {'value': 'pIC50', 'smiles': 'SMILES', 'ChEMBL_Compound_ID': 'ChEMBL_Compound_ID'}

for dataset in tqdm(set_list):
    dataset_path = result_dir + dataset + '/'
    dataset_df = pd.read_csv(csv_dir + dataset + '.csv')
#     dataset_df = dataset_df.rename(columns=new_column_names)
    # Loop through input files
    for input_file in os.listdir(dataset_path):
        if input_file[-5:] == 'pdbqt':
            # 读取文本文件
            file_name = input_file[:-6]
            cpd = file_name.split('_')[0]
            pocket = file_name.split('_')[-1]
            txt_path = dataset_path + file_name + '.txt'
            with open(txt_path, 'r') as file:
                lines = file.readlines()

            # 提取符合模式的行，并保存到列表中
            for line in lines:
                match_ = re.match(pattern, line)
                match__ = re.match(pattern_, line)
                match = match_ if match_ else match__
                if match:
                    number = int(match.group(1))
                    affinity = float(match.group().rsplit(maxsplit=1)[-1])
                    # 设置条件
                    condition = (dataset_df['ChEMBL_Compound_ID'] == cpd)  # 示例条件，可根据实际情况修改
                    # 根据条件筛选满足条件的行索引
                    row_indices = dataset_df[condition].index
                    column_name = f'Pocket_{pocket}-{number}_Vina_Score'  # 列名，根据文件名索引生成
                    dataset_df.loc[row_indices, column_name] = affinity

    # 保存更新后的CSV表格
    dataset_df.to_csv(output_path + dataset + '.csv', index=False)


100%|███████████████████████████████████████████████████████████████████████████| 6/6 [07:21<00:00, 73.52s/it]


# Do QVina-W on a batch of ligands and pockets in one protein

In [ ]:
import os
from tqdm import tqdm

pocket_num = 7
receptor_file = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/protein/pdb1boz.pdbqt'
ligand_pdbqt_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/ligand/pdbqt/'
config_file_template = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/protein/qvina_pocket_{}.txt'
output_root = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/qvina_pocket/'
qvina_w_path = '/home/yueming/Drug_Discovery/Baselines/qvina'
input_sets = os.listdir(ligand_pdbqt_dir)
# 循环执行命令
for set_name in tqdm(input_sets):
    output_dir = output_root + set_name + '/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    input_files = os.listdir(ligand_pdbqt_dir + set_name)
    for input_file in tqdm(input_files):
        if not os.path.exists(f'{output_dir}/{input_file}'):
            input_file_path = ligand_pdbqt_dir + set_name + '/' + input_file
            # 更改配置文件中的 receptor 路径为当前的 input_file
            for i in range(pocket_num):
                pocket = i + 1
                config_file = config_file_template.format(pocket)
                with open(config_file, 'r') as f:
                    config_lines = f.readlines()
                config_lines[0] = f'receptor = {receptor_file}\n'
                config_lines[1] = f'ligand = {ligand_pdbqt_dir + set_name}/{input_file}\n'
                config_lines[2] = f'out  = {output_dir}/{input_file[:-6]}_Pocket_{pocket}.pdbqt\n'
                config_lines[3] = f'log  = {output_dir}/{input_file[:-6]}_Pocket_{pocket}.txt\n'
                with open(config_file, 'w') as f:
                    f.writelines(config_lines)

                # 执行命令
                command = f'{qvina_w_path}/qvina-w_serial --config {config_file}'
                os.system(command)


# Read and save docking results for QVina-W

In [ ]:
import os, re
from tqdm import tqdm
from rdkit import Chem
import pandas as pd

# List of input SDF files
result_root = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/qvina_pocket/'
output_csv_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/qvina_pocket/'
input_sets = os.listdir(result_root)

# 定义正则表达式模式，只取top-1结果
pattern = r'^\s+1\s+(-?\d+\.\d+)'
pattern_ = r'^\s+1\s+(-?\d+)'

def extract_smiles_from_pdbqt(file_path):
    smiles = None
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('REMARK SMILES'):
                smiles = line.split('REMARK SMILES')[1].strip()
                break
    return smiles

for set_name in tqdm(input_sets):
    result_dir = result_root + set_name + '/'
    # 创建一个空的 DataFrame
    df = pd.DataFrame(columns=['ChEMBL_Compound_ID', 'SMILES', 'Vina_Score_1', 'Vina_Score_2', 'Vina_Score_3', 'Vina_Score_4', 
                               'Vina_Score_5', 'Vina_Score_6', 'Vina_Score_7'])
    for input_file in os.listdir(result_dir):
        if input_file[-5:] == 'pdbqt':
            file_name = input_file[:-6]
            cpd = file_name.split('_')[0]
            pocket = file_name.split('_')[-1]
            # 读取文本文件
            txt_path = result_dir + file_name + '.txt' # need to be coverted to "input_file.replace('pdbqt', 'txt')" after being corrected
            with open(txt_path, 'r') as txt:
                lines = txt.readlines()
            smiles = extract_smiles_from_pdbqt(result_dir + input_file)
            # 提取符合模式的行，并保存到列表中
            for line in lines:
                match = re.findall(pattern, line, flags=re.MULTILINE)
                if match:
                    affinity = float(match[0])
                    break
            if cpd in df['ChEMBL_Compound_ID'].values:
                condition = (df['ChEMBL_Compound_ID']==cpd)
                df.loc[condition, f'Vina_Score_{pocket}'] = affinity
            else:
                init_content_list = [cpd, smiles] + [None] * 7
                df.loc[len(df)] = init_content_list
                df.loc[len(df)-1, f'Vina_Score_{pocket}'] = affinity

    # 保存更新后的CSV表格
    df.to_csv(output_csv_dir + f'{set_name}/vina_results.csv', index=False)

# Do QVina-W on a batch of ligands and one protein

In [ ]:
import os
from tqdm import tqdm

receptor_file = '/home/yueming/Drug_Discovery/Baselines/qvina/dataset/6yg9.pdbqt'
ligand_pdbqt_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/ligand/pdbqt/'
config_file = '/home/yueming/Drug_Discovery/Baselines/qvina/dataset/6yg9_config.txt'
output_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/6yg9/'
qvina_w_path = '/home/yueming/Drug_Discovery/Baselines/qvina'
input_sets = os.listdir(ligand_pdbqt_dir)
# 循环执行命令
for set_name in tqdm(input_sets):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    input_files = os.listdir(ligand_pdbqt_dir + set_name)
    for input_file in tqdm(input_files):
        if not os.path.exists(f'{output_dir}/{input_file}'):
            input_file_path = ligand_pdbqt_dir + set_name + '/' + input_file
            # 更改配置文件中的 receptor 路径为当前的 input_file
            with open(config_file, 'r') as f:
                config_lines = f.readlines()
            config_lines[0] = f'receptor = {receptor_file}\n'
            config_lines[1] = f'ligand = {ligand_pdbqt_dir + set_name}/{input_file}\n'
            config_lines[2] = f'out  = {output_dir}/{input_file}\n'
            config_lines[3] = f'log  = {output_dir}/{input_file[:-6]}.txt\n'
            with open(config_file, 'w') as f:
                f.writelines(config_lines)

            # 执行命令
            command = f'{qvina_w_path}/qvina-w_serial --config {config_file}'
            os.system(command)


# Read and save docking results for QVina-W

In [ ]:
import os, re
from tqdm import tqdm
from rdkit import Chem
import pandas as pd

# List of input SDF files
result_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/6yg9/'
output_csv_path = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/6yg9/qvina_w_results.csv'
    
# 定义正则表达式模式
pattern = r'^\s+(\d+)\s+(-?\d+\.\d+)'
pattern_ = r'^\s+(\d+)\s+(-?\d+)'
# 创建一个空的 DataFrame
df = pd.DataFrame(columns=['ChEMBL_Compound_ID', 'SMILES', 'QVina-W_Score_1', 'QVina-W_Score_2', 'QVina-W_Score_3', 'QVina-W_Score_4', 
                           'QVina-W_Score_5', 'QVina-W_Score_6', 'QVina-W_Score_7', 'QVina-W_Score_8', 'QVina-W_Score_9'])

def extract_smiles_from_pdbqt(file_path):
    smiles = None
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('REMARK SMILES'):
                smiles = line.split('REMARK SMILES')[1].strip()
                break
    return smiles

def add_row_to_dataframe(df, content_list):
    num_columns = len(df.columns)
    if len(content_list) < num_columns:
        content_list.extend([None] * (num_columns - len(content_list)))
    df.loc[len(df)] = content_list

for input_file in tqdm(os.listdir(result_dir)):
    if input_file[-5:] == 'pdbqt':
        # 读取文本文件
        txt_path = result_dir + input_file.replace('pdbqt', 'txt')
        with open(txt_path, 'r') as txt:
            lines = txt.readlines()
        smiles = extract_smiles_from_pdbqt(result_dir + input_file)
        cid = input_file[:-6]  # 文件名索引
        content_list = [cid, smiles]
        # 提取符合模式的行，并保存到列表中
        for line in lines:
            match_ = re.match(pattern, line)
            match__ = re.match(pattern_, line)
            match = match_ if match_ else match__
            if match:
                number = int(match.group(1))
                affinity = float(match.group().rsplit(maxsplit=1)[-1])
                content_list.append(affinity)
        add_row_to_dataframe(df, content_list)

# 保存更新后的CSV表格
df.to_csv(output_csv_path, index=False)


# Preprocessing docking results to mols

In [1]:
# conda activate rdkit
import os
import pickle
from rdkit import Chem
import pandas as pd
from tqdm import tqdm
import pymol
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def generate_pocket(data_dir, protein_path, distance=5, save_states=1):
    complex_id = os.listdir(data_dir)
    for file in complex_id:
        if file[-6:] == '.pdbqt':
            file_name = file[:-6]
            cid = file_name.split('_')[0]
            pocket = file_name.split('_')[-1]
            pymol.cmd.delete('all')
            lig_path = os.path.join(data_dir, file)
            protein_name = protein_path.split('/')[-1].split('.')[0]

            pymol.cmd.load(lig_path)
            pymol.cmd.remove('hydrogens')
            pymol.cmd.load(protein_path)
            pymol.cmd.remove('resn HOH')
            object_list = pymol.cmd.get_object_list()  # 获取所有对象列表
            obj_ligand, obj_protein = object_list[0], object_list[1]
            total_states = pymol.cmd.count_states(obj_ligand)
            num_states = min(save_states, total_states)
            
            old_path = os.path.join(data_dir, f'{cid}_Pocket_{pocket}_{distance}A.pdb')
            if os.path.exists(old_path):
                os.rename(old_path, os.path.join(data_dir, f'{cid}_Pocket_{pocket}_Pose_1_{distance}A.pdb'))
            for state in range(1, num_states + 1):
                pocket_path = os.path.join(data_dir, f'{cid}_Pocket_{pocket}_Pose_{state}_{distance}A.pdb')
                if not os.path.exists(pocket_path) or recreate:
                    pymol.cmd.create(f"state_{state}", obj_ligand, state, 1)
                    pymol.cmd.select('Pocket', f'byres {protein_name} within {distance} of state_{state}')
                    pymol.cmd.save(pocket_path, 'Pocket')


def split_ligand_to_pdb(pdbqt_file, save_states=1):
    pymol.cmd.delete('all')
    pdb_name = pdbqt_file.split('/')[-1].split('.')[0]
    pymol.cmd.load(pdbqt_file)
    pymol.cmd.remove('hydrogens')
    object_list = pymol.cmd.get_object_list()  # 获取所有对象列表
    obj_ligand = object_list[0]
    total_states = pymol.cmd.count_states(obj_ligand)
    old_path = pdbqt_file.split('.')[0] + '_Pose_0.pdb'
    if os.path.exists(old_path):
        os.remove(old_path)
    num_states = min(save_states, total_states)
    for state in range(1, num_states + 1):
        save_path = pdbqt_file.split('.')[0] + f'_Pose_{state}.pdb'
        if not os.path.exists(save_path) or recreate:
            pymol.cmd.create(f"state_{state}", obj_ligand, state, 1)
            pymol.cmd.save(save_path, f"state_{state}")
                    
                
def generate_complex(data_dir, distance=5, input_ligand_format='pdbqt', save_states=1):
    file_list = os.listdir(data_dir)
    pdbqt_list = [x for x in file_list if x.split('.')[-1] == 'pdbqt']
    pbar = tqdm(total=len(pdbqt_list)*save_states)
    for i, pdbqt_file in enumerate(pdbqt_list):
        file_name = pdbqt_file[:-6]
        cid = file_name.split('_')[0]
        pocket = file_name.split('_')[-1]
        ligand_input_path = os.path.join(data_dir, f'{cid}_Pocket_{pocket}.{input_ligand_format}')
        # 调用函数，传入要处理的 PDBQT 文件名
        split_ligand_to_pdb(ligand_input_path, save_states=save_states)
        old_path = os.path.join(data_dir, f"{cid}_Complex_{pocket}_{distance}A.rdkit")
        if os.path.exists(old_path):
            os.rename(old_path, os.path.join(data_dir, f"{cid}_Complex_{pocket}_1_{distance}A.rdkit"))
        for pose in range(1, save_states + 1):
            old_path = os.path.join(data_dir, f"{cid}_Complex_{pocket}_{pose}_{distance}A.rdkit")
            if os.path.exists(old_path):
                os.rename(old_path, os.path.join(data_dir, f"{cid}_Complex_{pocket}-{pose}_{distance}A.rdkit"))
            save_path = os.path.join(data_dir, f"{cid}_Complex_{pocket}-{pose}_{distance}A.rdkit")
            if not os.path.exists(save_path) or recreate:
                pocket_path = os.path.join(data_dir, f'{cid}_Pocket_{pocket}_Pose_{pose}_{distance}A.pdb')
                ligand_path = ligand_input_path.split('.')[0] + f'_Pose_{pose}.pdb'
                if not os.path.exists(ligand_input_path):
                    if os.path.exists(ligand_path):
                        os.remove(ligand_path)
                    continue
                if not os.path.exists(ligand_path):
                    print(f'Not found pose: {ligand_file}')
                    continue
                
                ligand_file = Chem.MolFromPDBFile(ligand_path, removeHs=True)
                if ligand_file == None:
                    print(f"Unable to process ligand of {cid}")
                    continue

                pocket_file = Chem.MolFromPDBFile(pocket_path, removeHs=True)
                if pocket_file == None:
                    print(f"Unable to process protein of {cid}")
                    continue

                complex = (ligand_file, pocket_file)
                with open(save_path, 'wb') as f:
                    pickle.dump(complex, f)

                pbar.update(1)
                
recreate = True
task_list = ['E3', 'I4', 'I5']
task_dict = {'I1': ('CHEMBL202', 'pIC50', '1boz', 7, 1), 'E3': ('CHEMBL235', 'pEC50', '1zgy', 4, 2), 
             'I5': ('CHEMBL279', 'pIC50', '1ywn',3, 3), 'I4': ('CHEMBL2971', 'pIC50', '3ugc', 3, 3), 
             'I3': ('CHEMBL333', 'pIC50', '1ck7', 6, 2), 'E1': ('CHEMBL3820', 'pEC50', '3f9m', 6 ,1), 
             'I2': ('CHEMBL3976', 'pIC50', '4ebb', 2, 4), 'E2': ('CHEMBL4422', 'pEC50', '5tzr', 3, 3)}
for task_id in task_list:
    protein_name, assay_type, pdb_name, pocket_num, pose_num = task_dict[task_id]
    distance, input_ligand_format = 5, 'pdbqt'
    protein_path = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/protein/{pdb_name}.pdbqt'
    data_root = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/{pdb_name}/vina/'
    set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
    for dataset in tqdm(set_list):
        data_df = pd.read_csv(os.path.join(data_root, f'{protein_name}_{assay_type}_{dataset}.csv'))
        dataset_path = data_root + dataset + '/'

        ## generate pocket within 5 Ångström around ligand 
        generate_pocket(data_dir=dataset_path, protein_path=protein_path, distance=distance, save_states=pose_num)
        generate_complex(dataset_path, distance=distance, input_ligand_format=input_ligand_format, save_states=pose_num)

  0%|          | 0/6 [00:00<?, ?it/s]

 PyMOL not running, entering library mode (experimental)



 58%|█████▊    | 2741/4760 [00:12<00:09, 214.15it/s]


 82%|████████▏ | 3831/4688 [00:22<00:04, 174.74it/s]


 90%|█████████ | 4275/4736 [00:25<00:02, 163.05it/s]


...

 57%|█████▋    | 8116/14121 [03:47<02:37, 38.08it/s]


 70%|██████▉   | 9865/14121 [04:34<01:53, 37.60it/s]


 82%|████████▏ | 11578/14121 [05:20<01:08, 37.27it/s]


 94%|█████████▍| 13288/14121 [06:07<00:22, 37.07it/s]


100%|██████████| 6/6 [1:12:52<00:00, 728.70s/it]


# Save GIGN graphs for each ligand

In [4]:
# conda activate base
import os, re
import pandas as pd
import numpy as np
import pickle
from scipy.spatial import distance_matrix
import multiprocessing
from itertools import repeat
import networkx as nx
import torch 
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit import RDLogger
from rdkit import Chem
from torch_geometric.data import Batch, Data
from tqdm import tqdm
import warnings
RDLogger.DisableLog('rdApp.*')
np.set_printoptions(threshold=np.inf)
warnings.filterwarnings('ignore')
from torch_geometric.data import Batch

# %%
def one_of_k_encoding(k, possible_values):
    if k not in possible_values:
        raise ValueError(f"{k} is not a valid value in {possible_values}")
    return [k == e for e in possible_values]


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def atom_features(mol, graph, atom_symbols=['C', 'N', 'O', 'S', 'F', 'P', 'Cl', 'Br', 'I'], explicit_H=True):

    for atom in mol.GetAtoms():
        results = one_of_k_encoding_unk(atom.GetSymbol(), atom_symbols + ['Unknown']) + \
                one_of_k_encoding_unk(atom.GetDegree(),[0, 1, 2, 3, 4, 5, 6]) + \
                one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6]) + \
                one_of_k_encoding_unk(atom.GetHybridization(), [
                    Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                    Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.
                                        SP3D, Chem.rdchem.HybridizationType.SP3D2
                    ]) + [atom.GetIsAromatic()]
        # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
        if explicit_H:
            results = results + one_of_k_encoding_unk(atom.GetTotalNumHs(),
                                                    [0, 1, 2, 3, 4])

        atom_feats = np.array(results).astype(np.float32)

        graph.add_node(atom.GetIdx(), feats=torch.from_numpy(atom_feats))

def get_edge_index(mol, graph):
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        graph.add_edge(i, j)

def mol2graph(mol):
    graph = nx.Graph()
    atom_features(mol, graph)
    get_edge_index(mol, graph)

    graph = graph.to_directed()
    x = torch.stack([feats['feats'] for n, feats in graph.nodes(data=True)])
    if not graph.edges(data=False):
        return [], [], True
    edge_index = torch.stack([torch.LongTensor((u, v)) for u, v in graph.edges(data=False)]).T

    return x, edge_index, False

def inter_graph(ligand, pocket, dis_threshold = 5.):
    atom_num_l = ligand.GetNumAtoms()
    atom_num_p = pocket.GetNumAtoms()

    graph_inter = nx.Graph()
    pos_l = ligand.GetConformers()[0].GetPositions()
    pos_p = pocket.GetConformers()[0].GetPositions()
    dis_matrix = distance_matrix(pos_l, pos_p)
    node_idx = np.where(dis_matrix < dis_threshold)
    for i, j in zip(node_idx[0], node_idx[1]):
        graph_inter.add_edge(i, j+atom_num_l) 

    graph_inter = graph_inter.to_directed()
    edge_index_inter = torch.stack([torch.LongTensor((u, v)) for u, v in graph_inter.edges(data=False)]).T

    return edge_index_inter

# %%
def mols2graphs(complex_path_list, label, vina_score_list, save_path, dis_threshold):
    data_list = []
    fail_path = []
    for i, complex_path in enumerate(complex_path_list):
        if os.path.exists(complex_path):
            with open(complex_path, 'rb') as f:
                ligand, pocket = pickle.load(f)
        else:
            print('Complex file not found:', complex_path)
            fail_path.append(complex_path)
            continue

        atom_num_l = ligand.GetNumAtoms()
        atom_num_p = pocket.GetNumAtoms()

        pos_l = torch.FloatTensor(ligand.GetConformers()[0].GetPositions())
        pos_p = torch.FloatTensor(pocket.GetConformers()[0].GetPositions())
        x_l, edge_index_l, fail_l = mol2graph(ligand)
        x_p, edge_index_p, fail_p = mol2graph(pocket)
        if fail_l or fail_p:
            print('Failed to read complex file:', complex_path)
            fail_path.append(complex_path)
            continue

        x = torch.cat([x_l, x_p], dim=0)
        edge_index_intra = torch.cat([edge_index_l, edge_index_p+atom_num_l], dim=-1)
        try:
            edge_index_inter = inter_graph(ligand, pocket, dis_threshold=dis_threshold)
        except:
            print('Failed to read complex edges:', complex_path)
            fail_path.append(complex_path)
            continue
            
        y = torch.FloatTensor([label])
        vina_score = torch.FloatTensor([vina_score_list[i]])
        pos = torch.concat([pos_l, pos_p], dim=0)
        split = torch.cat([torch.zeros((atom_num_l, )), torch.ones((atom_num_p,))], dim=0)
        pocket = complex_path.split('/')[-1].split('_')[2]
        
        data = Data(x=x, edge_index_intra=edge_index_intra, edge_index_inter=edge_index_inter, y=y, vina_score=vina_score, pos=pos, pocket=pocket, split=split)
        data_list.append(data)
        
    if len(fail_path) == len(complex_path_list):
        return complex_path_list
    else:
        merged_data = Batch.from_data_list(data_list)
        torch.save(merged_data, save_path)
        return fail_path

# %%
class PLIDataLoader(DataLoader):
    def __init__(self, data, **kwargs):
        super().__init__(data, collate_fn=data.collate_fn, **kwargs)

class GraphDataset(Dataset):
    """
    This class is used for generating graph objects using multi process
    """
    def __init__(self, data_dir, data_df, dis_threshold=5, num_pose=1, graph_type='Graph_GIGN', assay_type='pIC50', num_process=8, create=False):
        self.data_dir = data_dir
        self.data_df = data_df
        self.dis_threshold = dis_threshold
        self.num_pose = num_pose
        self.graph_type = graph_type
        self.create = create
        self.graph_paths = None
        self.complex_ids = None
        self.assay_type = assay_type
        self.num_process = num_process
        self.mean, self.std = 0, 1
        self._pre_process()

    def _pre_process(self):
        data_dir = self.data_dir
        data_df = self.data_df
        graph_type = self.graph_type
        pocket_num = len(os.listdir(data_dir))

        complex_path_list, complex_id_list, pIC50_list, score_list, graph_path_list, dis_threshold_list = [], [], [], [], [], []
        file_list = os.listdir(data_dir)
        pocket_list_all = [x for x in file_list if x.split('_')[-1] == f'{self.dis_threshold}A.rdkit']
        not_found_list = []
        for i, row in data_df.iterrows():
            cid, pIC50 = row['ChEMBL_Compound_ID'], float(row[self.assay_type])
            complex_path_list_cid, complex_id_list_cid, score_list_cid = [], [], []
            graph_path = os.path.join(data_dir, f"{cid}_{graph_type}_{self.dis_threshold}A.pyg")
            pocket_list = [x for x in pocket_list_all if x.split('_')[0] == cid]
            if pocket_list:
                for pocket in pocket_list:
                    pocket_idx = pocket.split('_')[2]
                    score = float(row[f'Pocket_{pocket_idx}_Vina_Score'])
                    complex_path = os.path.join(data_dir, f"{cid}_Complex_{pocket_idx}_{self.dis_threshold}A.rdkit")
                    if len(pocket_idx.split('-')) > 1 and int(pocket_idx.split('-')[1]) <= self.num_pose or len(pocket_idx.split('-')) == 1:
                        complex_path_list_cid.append(complex_path)
                        score_list_cid.append(score)
                complex_path_list.append(complex_path_list_cid)
                score_list.append(score_list_cid)
                complex_id_list.append(cid)
                pIC50_list.append(pIC50)
                graph_path_list.append(graph_path)
                dis_threshold_list.append(self.dis_threshold)
            else:  
                not_found_list.append(graph_path)

        self.mean, self.std = np.mean(pIC50_list), np.std(pIC50_list)
        if self.create:
            print('Generate complex graph...')
            # multi-thread processing
            pool = multiprocessing.Pool(self.num_process)
            for complex_path, pIC50, vina_score ,graph_path, dis_threshold in zip(complex_path_list, pIC50_list, score_list, graph_path_list, dis_threshold_list):
                not_found_path = mols2graphs(complex_path, pIC50, vina_score ,graph_path, dis_threshold)
                if len(not_found_path) == len(complex_path):
                    not_found_list.append(graph_path)
            with open(data_dir + f'/not_found_list.pkl', 'wb') as f:
                pickle.dump(not_found_list, f)
            pool.close()
            pool.join()
        
        with open(data_dir + '/not_found_list.pkl', 'rb') as f:
            not_found_list = pickle.load(f)
        self.complex_ids = complex_id_list
        self.graph_paths = [x for x in graph_path_list if x not in not_found_list]

    def __getitem__(self, idx):
        data = torch.load(self.graph_paths[idx])
        match = re.search(r'CHEMBL(\d+)_', self.graph_paths[idx])
        chembl_id = match.group(0)
        data['idx'] = chembl_id[:-1]
        return data

    def collate_fn(self, batch):
        return Batch.from_data_list(batch)

    def __len__(self):
        return len(self.graph_paths)

task_list = ['E3', 'I4', 'I5']
task_dict = {'I1': ('CHEMBL202', 'pIC50', '1boz', 7, 1), 'E3': ('CHEMBL235', 'pEC50', '1zgy', 4, 2), 
             'I5': ('CHEMBL279', 'pIC50', '1ywn',3, 3), 'I4': ('CHEMBL2971', 'pIC50', '3ugc', 3, 3), 
             'I3': ('CHEMBL333', 'pIC50', '1ck7', 6, 2), 'E1': ('CHEMBL3820', 'pEC50', '3f9m', 6 ,1), 
             'I2': ('CHEMBL3976', 'pIC50', '4ebb', 2, 4), 'E2': ('CHEMBL4422', 'pEC50', '5tzr', 3, 3)}
for task_id in task_list:
    protein_name, assay_type, pdb_name, pocket_num, pose_num = task_dict[task_id]
    data_root = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/{pdb_name}/vina/'
    set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
    for set_name in set_list:
        data_dir = data_root + set_name + '/'
        data_df = pd.read_csv(os.path.join(data_root, f'{protein_name}_{assay_type}_{set_name}.csv'))
        dataset = GraphDataset(data_dir, data_df, graph_type='Graph_GIGN', assay_type=assay_type, dis_threshold=5, create=True)
        print('Dataset size:', len(dataset))
        data_loader = PLIDataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)
        for data in data_loader:
            # print(data) --> DataBatch(x=[2481, 35], y=[8], pos=[2481, 3], edge_index_intra=[2, 4884], edge_index_inter=[2, 4456], split=[2481], pocket=[8], batch=[2481], ptr=[9])
            data, pocket, idx, label, socre = data, data.pocket, data.idx, data.y, data.vina_score
            print(f'Loading {len(pocket)} data successfully')

Generate complex graph...
Dataset size: 162
Loading 162 data successfully
Generate complex graph...
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_5_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_6_5A.rdkit
Failed

Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL1800948_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_6_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_5_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_7_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vin

Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_7_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_5_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vin

Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_7_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_6_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vin

# Save GIGN graphs for each ligand with bond features

In [8]:
# conda activate base
import os, re
import pandas as pd
import numpy as np
import pickle
from scipy.spatial import distance_matrix
import multiprocessing
from itertools import repeat
import networkx as nx
import torch 
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit import RDLogger
from rdkit import Chem
from torch_geometric.data import Batch, Data
from tqdm import tqdm
import warnings
RDLogger.DisableLog('rdApp.*')
np.set_printoptions(threshold=np.inf)
warnings.filterwarnings('ignore')
from torch_geometric.data import Batch

graph_type = ['Graph_GIGN', 'Graph_Bond'][1]
task_list = ['I1']
task_dict = {'I1': ('CHEMBL202', 'pIC50', '1boz', 7, 1), 'E3': ('CHEMBL235', 'pEC50', '1zgy', 4, 2), 
             'I5': ('CHEMBL279', 'pIC50', '1ywn',3, 3), 'I4': ('CHEMBL2971', 'pIC50', '3ugc', 3, 3), 
             'I3': ('CHEMBL333', 'pIC50', '1ck7', 6, 2), 'E1': ('CHEMBL3820', 'pEC50', '3f9m', 6 ,1), 
             'I2': ('CHEMBL3976', 'pIC50', '4ebb', 2, 4), 'E2': ('CHEMBL4422', 'pEC50', '5tzr', 3, 3)}

# %%
def one_of_k_encoding(k, possible_values):
    if k not in possible_values:
        raise ValueError(f"{k} is not a valid value in {possible_values}")
    return [k == e for e in possible_values]


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def atom_features(mol, graph, atom_symbols=['C', 'N', 'O', 'S', 'F', 'P', 'Cl', 'Br', 'I'], explicit_H=True):

    for atom in mol.GetAtoms():
        results = one_of_k_encoding_unk(atom.GetSymbol(), atom_symbols + ['Unknown']) + \
                one_of_k_encoding_unk(atom.GetDegree(),[0, 1, 2, 3, 4, 5, 6]) + \
                one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6]) + \
                one_of_k_encoding_unk(atom.GetHybridization(), [
                    Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                    Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.
                                        SP3D, Chem.rdchem.HybridizationType.SP3D2
                    ]) + [atom.GetIsAromatic()]
        # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
        if explicit_H:
            results = results + one_of_k_encoding_unk(atom.GetTotalNumHs(),
                                                    [0, 1, 2, 3, 4])

        atom_feats = np.array(results).astype(np.float32)

        graph.add_node(atom.GetIdx(), feats=torch.from_numpy(atom_feats))

def bond_features(bond, use_chirality=True):
    bt = bond.GetBondType()
    bond_feats = [
        bt == Chem.rdchem.BondType.SINGLE, bt == Chem.rdchem.BondType.DOUBLE,
        bt == Chem.rdchem.BondType.TRIPLE, bt == Chem.rdchem.BondType.AROMATIC,
        bond.GetIsConjugated(),
        bond.IsInRing()
    ]
    if use_chirality:
        bond_feats = bond_feats + one_of_k_encoding_unk(
            str(bond.GetStereo()),
            ["STEREONONE", "STEREOANY", "STEREOZ", "STEREOE"])
    return np.array(bond_feats)
        
def get_edge_index(mol, graph):
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        bond_feats = bond_features(bond)
        graph.add_edge(i, j, weight=bond_feats)

def mol2graph(mol):
    graph = nx.Graph()
    atom_features(mol, graph)
    get_edge_index(mol, graph)

    graph = graph.to_directed()
    x = torch.stack([feats['feats'] for n, feats in graph.nodes(data=True)])
    x_bond = torch.tensor([graph[u][v]['weight'] for u, v in graph.edges()])
    if not graph.edges(data=False):
        return [], [], [], True
    edge_index = torch.stack([torch.LongTensor((u, v)) for u, v in graph.edges(data=False)]).T

    return x, x_bond, edge_index, False

def inter_graph(ligand, pocket, dis_threshold = 5.):
    atom_num_l = ligand.GetNumAtoms()
    atom_num_p = pocket.GetNumAtoms()

    graph_inter = nx.Graph()
    pos_l = ligand.GetConformers()[0].GetPositions()
    pos_p = pocket.GetConformers()[0].GetPositions()
    dis_matrix = distance_matrix(pos_l, pos_p)
    node_idx = np.where(dis_matrix < dis_threshold)
    for i, j in zip(node_idx[0], node_idx[1]):
        graph_inter.add_edge(i, j+atom_num_l) 

    graph_inter = graph_inter.to_directed()
    edge_index_inter = torch.stack([torch.LongTensor((u, v)) for u, v in graph_inter.edges(data=False)]).T

    return edge_index_inter

# %%
def mols2graphs(complex_path_list, label, vina_score_list, save_path, dis_threshold):
    data_list = []
    fail_path = []
    for i, complex_path in enumerate(complex_path_list):
        if os.path.exists(complex_path):
            with open(complex_path, 'rb') as f:
                ligand, pocket = pickle.load(f)
        else:
            print('Complex file not found:', complex_path)
            fail_path.append(complex_path)
            continue

        atom_num_l = ligand.GetNumAtoms()
        atom_num_p = pocket.GetNumAtoms()

        pos_l = torch.FloatTensor(ligand.GetConformers()[0].GetPositions())
        pos_p = torch.FloatTensor(pocket.GetConformers()[0].GetPositions())
        x_l, x_l_bond, edge_index_l, fail_l = mol2graph(ligand)
        x_p, x_p_bond, edge_index_p, fail_p = mol2graph(pocket)
        if fail_l or fail_p:
            print('Failed to read complex file:', complex_path)
            fail_path.append(complex_path)
            continue

        x = torch.cat([x_l, x_p], dim=0)
        x_bond = torch.cat([x_l_bond, x_p_bond], dim=0)
        edge_index_intra = torch.cat([edge_index_l, edge_index_p+atom_num_l], dim=-1)
        try:
            edge_index_inter = inter_graph(ligand, pocket, dis_threshold=dis_threshold)
        except:
            print('Failed to read complex edges:', complex_path)
            fail_path.append(complex_path)
            continue
            
        y = torch.FloatTensor([label])
        vina_score = torch.FloatTensor([vina_score_list[i]])
        pos = torch.concat([pos_l, pos_p], dim=0)
        split = torch.cat([torch.zeros((atom_num_l, )), torch.ones((atom_num_p,))], dim=0)
        pocket = complex_path.split('/')[-1].split('_')[2]
        
        data = Data(x=x, x_bond=x_bond, edge_index_intra=edge_index_intra, edge_index_inter=edge_index_inter, y=y, vina_score=vina_score, pos=pos, pocket=pocket, split=split)
        data_list.append(data)
        
    if len(fail_path) == len(complex_path_list):
        return complex_path_list
    else:
        merged_data = Batch.from_data_list(data_list)
        torch.save(merged_data, save_path)
        return fail_path

# %%
class PLIDataLoader(DataLoader):
    def __init__(self, data, **kwargs):
        super().__init__(data, collate_fn=data.collate_fn, **kwargs)

class GraphDataset(Dataset):
    """
    This class is used for generating graph objects using multi process
    """
    def __init__(self, data_dir, data_df, dis_threshold=5, num_pose=1, graph_type='Graph_GIGN', assay_type='pIC50', num_process=8, create=False):
        self.data_dir = data_dir
        self.data_df = data_df
        self.dis_threshold = dis_threshold
        self.num_pose = num_pose
        self.graph_type = graph_type
        self.create = create
        self.graph_paths = None
        self.complex_ids = None
        self.assay_type = assay_type
        self.num_process = num_process
        self.mean, self.std = 0, 1
        self._pre_process()

    def _pre_process(self):
        data_dir = self.data_dir
        data_df = self.data_df
        graph_type = self.graph_type
        pocket_num = len(os.listdir(data_dir))

        complex_path_list, complex_id_list, pIC50_list, score_list, graph_path_list, dis_threshold_list = [], [], [], [], [], []
        file_list = os.listdir(data_dir)
        pocket_list_all = [x for x in file_list if x.split('_')[-1] == f'{self.dis_threshold}A.rdkit']
        not_found_list = []
        for i, row in data_df.iterrows():
            cid, pIC50 = row['ChEMBL_Compound_ID'], float(row[self.assay_type])
            complex_path_list_cid, complex_id_list_cid, score_list_cid = [], [], []
            graph_path = os.path.join(data_dir, f"{cid}_{graph_type}_{self.dis_threshold}A.pyg")
            pocket_list = [x for x in pocket_list_all if x.split('_')[0] == cid]
            if pocket_list:
                for pocket in pocket_list:
                    pocket_idx = pocket.split('_')[2]
                    score = float(row[f'Pocket_{pocket_idx}_Vina_Score'])
                    complex_path = os.path.join(data_dir, f"{cid}_Complex_{pocket_idx}_{self.dis_threshold}A.rdkit")
                    if len(pocket_idx.split('-')) > 1 and int(pocket_idx.split('-')[1]) <= self.num_pose or len(pocket_idx.split('-')) == 1:
                        complex_path_list_cid.append(complex_path)
                        score_list_cid.append(score)
                complex_path_list.append(complex_path_list_cid)
                score_list.append(score_list_cid)
                complex_id_list.append(cid)
                pIC50_list.append(pIC50)
                graph_path_list.append(graph_path)
                dis_threshold_list.append(self.dis_threshold)
            else:  
                not_found_list.append(graph_path)

        self.mean, self.std = np.mean(pIC50_list), np.std(pIC50_list)
        if self.create:
            print('Generate complex graph...')
            # multi-thread processing
            pool = multiprocessing.Pool(self.num_process)
            for complex_path, pIC50, vina_score ,graph_path, dis_threshold in zip(complex_path_list, pIC50_list, score_list, graph_path_list, dis_threshold_list):
                not_found_path = mols2graphs(complex_path, pIC50, vina_score ,graph_path, dis_threshold)
                if len(not_found_path) == len(complex_path):
                    not_found_list.append(graph_path)
            with open(data_dir + f'/not_found_list.pkl', 'wb') as f:
                pickle.dump(not_found_list, f)
            pool.close()
            pool.join()
        
        with open(data_dir + '/not_found_list.pkl', 'rb') as f:
            not_found_list = pickle.load(f)
        self.complex_ids = complex_id_list
        self.graph_paths = [x for x in graph_path_list if x not in not_found_list]

    def __getitem__(self, idx):
        data = torch.load(self.graph_paths[idx])
        match = re.search(r'CHEMBL(\d+)_', self.graph_paths[idx])
        chembl_id = match.group(0)
        data['idx'] = chembl_id[:-1]
        return data

    def collate_fn(self, batch):
        return Batch.from_data_list(batch)

    def __len__(self):
        return len(self.graph_paths)

for task_id in task_list:
    protein_name, assay_type, pdb_name, pocket_num, pose_num = task_dict[task_id]
    data_root = f'/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/{protein_name}/{pdb_name}/vina/'
    set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
    for set_name in set_list:
        data_dir = data_root + set_name + '/'
        data_df = pd.read_csv(os.path.join(data_root, f'{protein_name}_{assay_type}_{set_name}.csv'))
        dataset = GraphDataset(data_dir, data_df, graph_type=graph_type, assay_type=assay_type, dis_threshold=5, create=True)
        print('Dataset size:', len(dataset))
        data_loader = PLIDataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)
        for data in data_loader:
            # print(data) --> DataBatch(x=[2481, 35], y=[8], pos=[2481, 3], edge_index_intra=[2, 4884], edge_index_inter=[2, 4456], split=[2481], pocket=[8], batch=[2481], ptr=[9])
            x, pocket, idx, label, socre, x_bond = data.x, data.pocket, data.idx, data.y, data.vina_score, data.x_bond
            print(f'Loading {len(pocket)} data successfully, atom feature dim: {x.shape}, bond feature dim: {x_bond.shape}')

Generate complex graph...
Dataset size: 162
Loading 162 data successfully, atom feature dim: torch.Size([328815, 35]), bond feature dim: torch.Size([646708, 10])
Generate complex graph...
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_5_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_1/CHEMBL4160822_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines

Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL1800948_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_6_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_5_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_7_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3943671_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vin

Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3912587_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3912587_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_6_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_2/CHEMBL3974488_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vin

Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824772_Complex_6_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_3_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_1_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_2_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_7_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vina/train_3/CHEMBL1824534_Complex_4_5A.rdkit
Failed to read complex file: /home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/vin

# Count pocket number for each ligand ready to read

In [ ]:
import os, json, re
from tqdm import tqdm

folder_path = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/6yg9'
save_file = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/6yg9.json'
count_dict, file_count = {}, 0

# 获取文件夹中所有文件
files = os.listdir(folder_path)

for file in tqdm(files):
    cid = file.split('_')[0]
    match = re.search(r'Graph_GIGN_Pocket_(\d+)_5A.pyg', file)
    if match:
        pocket_idx = int(match.group(1))
        # 检查文件名是否包含目标 ID
        if cid in count_dict.keys():
            count_dict[cid].append(pocket_idx)
        else:
            count_dict[cid] = [pocket_idx]

with open(save_file, 'w') as f:
    json.dump(count_dict, f, indent=4, separators=(",", ": "))

# Do QVina-W on ligands with activity

In [ ]:
import subprocess
import pandas as pd
from tqdm import tqdm

# 定义输入输出文件路径列表
receptor_file = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/protein/pdb1boz.pdbqt'
input_dir_ligand = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/ligand/pdbqt/'
output_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/qvina/'
config_file = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/protein/1boz_config.txt'
qvina_w_path = '/home/yueming/Drug_Discovery/Baselines/qvina'
input_sets = os.listdir(input_dir_ligand)
# 循环处理每个文件路径
for set_name in tqdm(input_sets):
    input_ligands = os.listdir(input_dir_ligand + set_name)
    for input_file in tqdm(input_ligands):
        if not os.path.exists(output_dir + set_name):
            os.makedirs(output_dir + set_name)
        if not os.path.exists(f'{output_dir + set_name}/{input_file}'):
            input_file_path = input_dir_ligand + set_name + '/' + input_file
            # 更改配置文件中的 receptor 路径为当前的 input_file
            with open(config_file, 'r') as f:
                config_lines = f.readlines()
            config_lines[0] = f'receptor = {receptor_file}\n'
            config_lines[1] = f'ligand = {input_dir_ligand + set_name}/{input_file}\n'
            config_lines[2] = f'out  = {output_dir + set_name}/{input_file}\n'
            config_lines[3] = f'log  = {output_dir + set_name}/{input_file[:-6]}.txt\n'
            with open(config_file, 'w') as f:
                f.writelines(config_lines)

        # 执行命令
        command = f'{qvina_w_path}/qvina-w_serial --config {config_file}'
        os.system(command)

# Read and save QVina-W results

In [ ]:
import os, re
from tqdm import tqdm
from rdkit import Chem
import pandas as pd

# List of input SDF files
result_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/qvina/'
data_sets = os.listdir(result_dir)
csv_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/activity/CHEMBL202_IC50_in_nM_'
output_file = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/qvina/CHEMBL202_pIC50_'
    
# 定义正则表达式模式
pattern = r'^\s+(\d+)\s+(-?\d+\.\d+)'
pattern_ = r'^\s+(\d+)\s+(-?\d+)'
new_column_names = {'value': 'pIC50', 'smiles': 'SMILES', 'ChEMBL_Compound_ID': 'ChEMBL_Compound_ID'}

for dataset in tqdm(data_sets):
    if dataset[-4:] != '.csv':
        dataset_path = result_dir + dataset + '/'
        dataset_df = pd.read_csv(csv_dir + dataset + '.csv')
        dataset_df = dataset_df.rename(columns=new_column_names)

        # Loop through input files
        for input_file in tqdm(os.listdir(dataset_path)):
            if input_file[-4:] == '.txt':
                # 读取文本文件
                file_path = dataset_path + input_file
                with open(file_path, 'r') as file:
                    lines = file.readlines()

                # 提取符合模式的行，并保存到列表中
                for line in lines:
                    match_ = re.match(pattern, line)
                    match__ = re.match(pattern_, line)
                    match = match_ if match_ else match__
                    if match:
                        number = int(match.group(1))
                        affinity = float(match.group().rsplit(maxsplit=1)[-1])
                        # 根据文件名索引，将数据添加到CSV表格中
                        file_index = input_file[:-4]  # 文件名索引
                        # 设置条件
                        condition = (dataset_df['ChEMBL_Compound_ID'] == file_index)  # 示例条件，可根据实际情况修改
                        # 根据条件筛选满足条件的行索引
                        row_indices = dataset_df[condition].index
                        column_name = f'Pocket_{number}_QVina-W_Score'  # 列名，根据文件名索引生成
                        dataset_df.loc[row_indices, column_name] = affinity

    # 保存更新后的CSV表格
    dataset_df.to_csv(output_file + dataset + '.csv', index=False)


# Prepare ground truth ligand files

In [ ]:
# Desktop workstation
import os, shutil
from rdkit import Chem

ground_dir = '/data1/Drug_Discovery/Docking/data/pdb_with_activity/CHEMBL202'
file_list = os.listdir(ground_dir)
set_name_list = [x for x in file_list if len(x.split(".")) < 2]
for set_name in set_name_list:
    set_path = os.path.join(ground_dir, set_name)
    set_file_list = os.listdir(set_path)
    pdb_list = [x.split(".")[0] for x in set_file_list if x.split(".")[-1] == 'pdb']
    sdf_list = [x for x in set_file_list if x.split(".")[-1] == 'sdf']
    for pdb in pdb_list:
        ligand_name = [elem for elem in sdf_list if pdb in elem]
        input_file_path = os.path.join(set_path, ligand_name[0])
        output_file_path = os.path.join(set_path, ligand_name[0].replace('sdf', 'pdbqt'))
        if os.path.exists(output_file_path):
            continue
        try:
            # Load the molecule from the SDF file
            mol = Chem.SDMolSupplier(input_file_path)[0]
            # Add explicit hydrogens
#             sanitizedmol_mol = Chem.SanitizeMol(mol)
#             mol = Chem.Kekulize(mol)
            mol = Chem.AddHs(mol)
        except:
            print(input_file_path)
            continue
        # Save the modified molecule with explicit hydrogens
        Chem.SDWriter(input_file_path).write(mol)
        command = f"mk_prepare_ligand.py -i {input_file_path} -o {output_file_path}"
        print(command)
        !{command}

# Preprocessing docking data

In [ ]:
# conda activate rdkit
import os
import pickle
from rdkit import Chem
import pandas as pd
from tqdm import tqdm
import pymol
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

def pymol_operations(lig_native_path, protein_path, pocket_prefix, pocket_suffix, pocket_id):
    pymol.cmd.delete('all')
    pymol.cmd.load(lig_native_path)
    pymol.cmd.remove('hydrogens')
    pymol.cmd.load(protein_path)
    pymol.cmd.remove('resn HOH')
    object_list = pymol.cmd.get_object_list()  # 获取所有对象列表
    obj_ligand, obj_protein = object_list[0], object_list[1]
    num_states = pymol.cmd.count_states(obj_ligand)

    for state in range(1, num_states + 1):
        pocket_path = pocket_prefix + f'{state}' + pocket_suffix if num_states > 1 else pocket_prefix + f'{pocket_id}' + pocket_suffix
        if not os.path.exists(pocket_path):
            pymol.cmd.create(f"state_{state}", obj_ligand, state, 1)
            pymol.cmd.select('Pocket', f'byres {obj_protein} within {distance} of state_{state}')
            pymol.cmd.save(pocket_path, 'Pocket')
            pymol.cmd.delete('Pocket')
        
def generate_pocket(data_dir, protein_path, ground_dir, distance=5):
    complex_id = os.listdir(data_dir)
    for file in complex_id:
        if file[-6:] == '.pdbqt':
            cid = file[:-6]
            lig_native_path = os.path.join(data_dir, file)
            protein_name = protein_path.split('/')[-1].split('.')[0]
            pocket_prefix = os.path.join(data_dir, f'{cid}_Pocket_')
            pocket_suffix = f'_{distance}A.pdb'
            pymol_operations(lig_native_path, protein_path, pocket_prefix, pocket_suffix, pocket_id)
                
    ground_files = os.listdir(ground_dir)
    ground_pdb_list = [x for x in ground_files if x.split(".")[-1] == 'pdb' and len(x.split("_")) < 2]
    ground_pdbqt_list = [x for x in ground_files if x.split(".")[-1] == 'pdbqt']
    ligand_counter = {}
    for pdb_file in ground_pdb_list:
        pdb = pdb_file.split(".")[0]
        ligand_name = [elem for elem in ground_pdbqt_list if pdb in elem]
        try:
            ligand_id = ligand_name[0].split(".")[0].split("_")[1]
        except:
            print(f'Cannot read pdbqt ligand of protein: {pdb}')
            continue
        ligand_counter[ligand_id] = ligand_counter[ligand_id] + 1 if ligand_id in ligand_counter.keys() else 1
        lig_native_path = os.path.join(ground_dir, ligand_name[0])
        protein_path = os.path.join(ground_dir, pdb_file)
        pocket_prefix = os.path.join(ground_dir, f'{ligand_id}_Pocket_G')
        pocket_suffix = f'_{pdb}_{distance}A.pdb'
        pocket_id = ligand_counter[ligand_id]
        pymol_operations(lig_native_path, protein_path, pocket_prefix, pocket_suffix, pocket_id)


def generate_complex(data_dir, distance=5, input_ligand_format='mol2'):
    file_list = os.listdir(data_dir)
    pdb_list = [x for x in file_list if x.split('_')[-1] == f'{distance}A.pdb']
    pdbqt_list = [x for x in file_list if x.split('.')[-1] == f'pdbqt']
    pbar = tqdm(total=len(pdb_list))
    for pdb_file in pdb_list:
        cid, pocket = pdb_file.split('_')[0], pdb_file.split('_')[2]
        pocket_path = data_dir + pdb_file
        ligand_name = [x for x in pdbqt_list if x.split('.')[0].split('_')[-1] == cid]
        ligand_input_path = os.path.join(data_dir, ligand_name[0])
        ligand_path = ligand_input_path.replace(f".{input_ligand_format}", ".pdb")
        if not os.path.exists(ligand_input_path):
            if os.path.exists(ligand_path):
                os.remove(ligand_path)
            continue
        if input_ligand_format != 'pdb' and not os.path.exists(ligand_path):
            os.system(f'obabel {ligand_input_path} -O {ligand_path} -d')
        else:
            ligand_path = os.path.join(data_dir, ligand_name[0].replace(f".{input_ligand_format}", ".pdb"))

        save_path = os.path.join(data_dir, f"{cid}_Complex_{pocket}_{distance}A.rdkit")
        ligand = Chem.MolFromPDBFile(ligand_path, removeHs=True)
        if ligand == None:
            print(f"Unable to process ligand of {cid}")
            continue
            
        pocket = Chem.MolFromPDBFile(pocket_path, removeHs=True)
        if pocket == None:
            print(f"Unable to process protein of {pocket_path}")
            continue

        complex = (ligand, pocket)
        with open(save_path, 'wb') as f:
            pickle.dump(complex, f)
        os.remove(ligand_path)
        pbar.update(1)

distance = 5
input_ligand_format = 'pdbqt'
protein_path = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/protein/pdb1boz.pdbqt'
ground_root = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/pdb_with_activity/'
data_root = './data/CHEMBL202/1boz/'
set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
for dataset in tqdm(set_list):
    data_df = pd.read_csv(os.path.join(data_root, f'CHEMBL202_pIC50_{dataset}.csv'))
    dataset_path = data_root + dataset + '/'
    ground_dir = ground_root + dataset + '/'
    ## generate pocket within 5 Ångström around ligand 
    generate_pocket(data_dir=dataset_path, protein_path=protein_path, ground_dir=ground_dir, distance=distance)
    generate_complex(data_dir=dataset_path, distance=distance, input_ligand_format=input_ligand_format)
    generate_complex(data_dir=ground_dir, distance=distance, input_ligand_format=input_ligand_format)

# Process activity and structure data for model reading

In [ ]:
# conda activate base
import os, re
import pandas as pd
import numpy as np
import pickle
from scipy.spatial import distance_matrix
import multiprocessing
from itertools import repeat
import networkx as nx
import torch 
from torch.utils.data import Dataset, DataLoader
from rdkit import Chem
from rdkit import RDLogger
from rdkit import Chem
from torch_geometric.data import Batch, Data
from tqdm import tqdm
import warnings
RDLogger.DisableLog('rdApp.*')
np.set_printoptions(threshold=np.inf)
warnings.filterwarnings('ignore')
from torch_geometric.data import Batch

# %%
def one_of_k_encoding(k, possible_values):
    if k not in possible_values:
        raise ValueError(f"{k} is not a valid value in {possible_values}")
    return [k == e for e in possible_values]


def one_of_k_encoding_unk(x, allowable_set):
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))


def atom_features(mol, graph, atom_symbols=['C', 'N', 'O', 'S', 'F', 'P', 'Cl', 'Br', 'I'], explicit_H=True):

    for atom in mol.GetAtoms():
        results = one_of_k_encoding_unk(atom.GetSymbol(), atom_symbols + ['Unknown']) + \
                one_of_k_encoding_unk(atom.GetDegree(),[0, 1, 2, 3, 4, 5, 6]) + \
                one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6]) + \
                one_of_k_encoding_unk(atom.GetHybridization(), [
                    Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                    Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.
                                        SP3D, Chem.rdchem.HybridizationType.SP3D2
                    ]) + [atom.GetIsAromatic()]
        # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
        if explicit_H:
            results = results + one_of_k_encoding_unk(atom.GetTotalNumHs(),
                                                    [0, 1, 2, 3, 4])

        atom_feats = np.array(results).astype(np.float32)

        graph.add_node(atom.GetIdx(), feats=torch.from_numpy(atom_feats))

def get_edge_index(mol, graph):
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        graph.add_edge(i, j)

def mol2graph(mol):
    graph = nx.Graph()
    atom_features(mol, graph)
    get_edge_index(mol, graph)

    graph = graph.to_directed()
    x = torch.stack([feats['feats'] for n, feats in graph.nodes(data=True)])
    if not graph.edges(data=False):
        return [], [], True
    edge_index = torch.stack([torch.LongTensor((u, v)) for u, v in graph.edges(data=False)]).T

    return x, edge_index, False

def inter_graph(ligand, pocket, dis_threshold = 5.):
    atom_num_l = ligand.GetNumAtoms()
    atom_num_p = pocket.GetNumAtoms()

    graph_inter = nx.Graph()
    pos_l = ligand.GetConformers()[0].GetPositions()
    pos_p = pocket.GetConformers()[0].GetPositions()
    dis_matrix = distance_matrix(pos_l, pos_p)
    node_idx = np.where(dis_matrix < dis_threshold)
    for i, j in zip(node_idx[0], node_idx[1]):
        graph_inter.add_edge(i, j+atom_num_l) 

    graph_inter = graph_inter.to_directed()
    edge_index_inter = torch.stack([torch.LongTensor((u, v)) for u, v in graph_inter.edges(data=False)]).T

    return edge_index_inter

# %%
def mols2graphs(complex_path_list, label, vina_score_list, save_path, dis_threshold):
    data_list = []
    fail_path = []
    for i, complex_path in enumerate(complex_path_list):
        if os.path.exists(complex_path):
            with open(complex_path, 'rb') as f:
                ligand, pocket = pickle.load(f)
        else:
            print('Complex file not found:', complex_path)
            fail_path.append(complex_path)
            continue

        atom_num_l = ligand.GetNumAtoms()
        atom_num_p = pocket.GetNumAtoms()

        pos_l = torch.FloatTensor(ligand.GetConformers()[0].GetPositions())
        pos_p = torch.FloatTensor(pocket.GetConformers()[0].GetPositions())
        x_l, edge_index_l, fail_l = mol2graph(ligand)
        x_p, edge_index_p, fail_p = mol2graph(pocket)
        if fail_l or fail_p:
            print('Failed to read complex file:', complex_path)
            fail_path.append(complex_path)
            continue

        x = torch.cat([x_l, x_p], dim=0)
        edge_index_intra = torch.cat([edge_index_l, edge_index_p+atom_num_l], dim=-1)
        try:
            edge_index_inter = inter_graph(ligand, pocket, dis_threshold=dis_threshold)
        except:
            print('Failed to read complex edges:', complex_path)
            fail_path.append(complex_path)
            continue
            
        y = torch.FloatTensor([label])
        vina_score = torch.FloatTensor([vina_score_list[i]])
        pos = torch.concat([pos_l, pos_p], dim=0)
        split = torch.cat([torch.zeros((atom_num_l, )), torch.ones((atom_num_p,))], dim=0)
        pocket = complex_path.split('/')[-1].split('_')[2]
        
        data = Data(x=x, edge_index_intra=edge_index_intra, edge_index_inter=edge_index_inter, y=y, vina_score=vina_score, pos=pos, pocket=pocket, split=split)
        data_list.append(data)
        
    if len(fail_path) == len(complex_path_list):
        return complex_path_list
    else:
        merged_data = Batch.from_data_list(data_list)
        torch.save(merged_data, save_path)
        return fail_path

# %%
class PLIDataLoader(DataLoader):
    def __init__(self, data, **kwargs):
        super().__init__(data, collate_fn=data.collate_fn, **kwargs)

class GraphDataset(Dataset):
    """
    This class is used for generating graph objects using multi process
    """
    def __init__(self, data_dir, ground_dir, data_df, dis_threshold=5, graph_type='Graph_GIGN', num_process=8, create=False):
        self.data_dir = data_dir
        self.ground_dir = ground_dir
        self.data_df = data_df
        self.dis_threshold = dis_threshold
        self.graph_type = graph_type
        self.create = create
        self.graph_paths = None
        self.complex_ids = None
        self.num_process = num_process
        self.mean, self.std = 0, 1
        self._pre_process()

    def _pre_process(self):
        data_dir = self.data_dir
        ground_dir = self.ground_dir
        data_df = self.data_df
        graph_type = self.graph_type

        complex_path_list, complex_id_list, pIC50_list, vina_score_list, graph_path_list, dis_threshold_list = [], [], [], [], [], []
        file_list = os.listdir(data_dir)
        pocket_list_all = [x for x in file_list if x.split('_')[-1] == f'{self.dis_threshold}A.rdkit']
        ground_file_list = os.listdir(ground_dir)
        ground_pocket_list_all = [x for x in ground_file_list if x.split('_')[-1] == f'{self.dis_threshold}A.rdkit']
        not_found_list = []
        for i, row in data_df.iterrows():
            cid, pIC50 = row['ChEMBL_Compound_ID'], float(row['pIC50'])
            complex_path_list_cid, complex_id_list_cid, pIC50_list_cid, vina_score_list_cid, graph_path_list_cid, dis_threshold_list_cid = [], [], [], [], [], []
            graph_path = os.path.join(data_dir, f"{cid}_{graph_type}_{self.dis_threshold}A.pyg")
            pocket_list = [x for x in pocket_list_all if x.split('_')[0] == cid]
            ground_pocket_list = [x for x in ground_pocket_list_all if x.split('_')[0] == cid]
            if pocket_list:
                for pocket in pocket_list:
                    pocket_idx = pocket.split('_')[2]
#                     if int(pocket_idx) <= 3:
                    complex_path = os.path.join(data_dir, f"{cid}_Complex_{pocket_idx}_{self.dis_threshold}A.rdkit")
                    complex_path_list_cid.append(complex_path)
                    vina_score_list_cid.append(row[f'Pocket_{pocket_idx}_QVina-W_Score'])
                if ground_pocket_list:
                    for pocket in ground_pocket_list:
                        pocket_idx = pocket.split('_')[2]
                        complex_path = os.path.join(ground_dir, f"{cid}_Complex_{pocket_idx}_{self.dis_threshold}A.rdkit")
                        complex_path_list_cid.append(complex_path)
                        vina_score_list_cid.append(-1e2)
                complex_path_list.append(complex_path_list_cid)
                vina_score_list.append(vina_score_list_cid)
                complex_id_list.append(cid)
                pIC50_list.append(pIC50)
                graph_path_list.append(graph_path)
                dis_threshold_list.append(self.dis_threshold)
            else:  
                not_found_list.append(graph_path)

        self.mean, self.std = np.mean(pIC50_list), np.std(pIC50_list)
        if self.create:
            print('Generate complex graph...')
            # multi-thread processing
            pool = multiprocessing.Pool(self.num_process)
            complex_paths_list = [len(x) for x in complex_path_list]
            vina_scores_list = [len(x) for x in vina_score_list]
            print(len(complex_paths_list), len(pIC50_list), len(vina_scores_list), len(graph_path_list), len(dis_threshold_list))
            for complex_path, pIC50, vina_score ,graph_path, dis_threshold in zip(complex_path_list, pIC50_list, vina_score_list, graph_path_list, dis_threshold_list):
                not_found_path = mols2graphs(complex_path, pIC50, vina_score ,graph_path, dis_threshold)
                if len(not_found_path) == len(complex_path):
                    not_found_list.append(graph_path)
            with open(data_dir + f'/not_found_list.pkl', 'wb') as f:
                pickle.dump(not_found_list, f)
            pool.close()
            pool.join()
        
        with open(data_dir + '/not_found_list.pkl', 'rb') as f:
            not_found_list = pickle.load(f)
        self.complex_ids = complex_id_list
        self.graph_paths = graph_path_list
        self.graph_paths = [x for x in graph_path_list if x not in not_found_list]

    def __getitem__(self, idx):
        data = torch.load(self.graph_paths[idx])
        match = re.search(r'CHEMBL(\d+)_', self.graph_paths[idx])
        chembl_id = match.group(0)
        data['idx'] = chembl_id[:-1]
        return data

    def collate_fn(self, batch):
        return Batch.from_data_list(batch)

    def __len__(self):
        return len(self.graph_paths)


data_root = './data/CHEMBL202/1boz/qvina/'
ground_root = './data/CHEMBL202/pdb_with_activity/'
set_list = ['test', 'train_1', 'train_2', 'train_3', 'train_4', 'train_5']
for dataset in tqdm(set_list):
    data_df = pd.read_csv(os.path.join(data_root, f'CHEMBL202_pIC50_{dataset}.csv'))
    dataset_path = data_root + dataset + '/'
    ground_path = ground_root + dataset + '/'
    data_set = GraphDataset(dataset_path, ground_path, data_df, graph_type='Graph_GIGN', dis_threshold=5, create=True)
    print(len(data_set))
    data_loader = PLIDataLoader(data_set, batch_size=10, shuffle=True, num_workers=4)
    for data in data_loader:
        # print(data) --> DataBatch(x=[2481, 35], y=[8], pos=[2481, 3], edge_index_intra=[2, 4884], edge_index_inter=[2, 4456], split=[2481], pocket=[8], batch=[2481], ptr=[9])
        data, pocket, idx, vina_score = data, data.pocket, data.idx, data.vina_score
        print(f'Loading {len(idx)} compounds with pockets: {pocket}')

# Assemble the same compound with different pockets

In [ ]:
from tqdm import tqdm
import shutil

data_dir = '/home/yueming/Drug_Discovery/Baselines/GIGN-main/GIGN/data/CHEMBL202/1boz/'
input_set_paths = os.listdir(data_dir)
input_sets = [x for x in input_set_paths if x.split('.')[-1] != 'pkl']
for set_name in tqdm(input_sets):
    set_path = data_dir + set_name + '/'
    pocket_names = os.listdir(set_path)
    for pocket_name in pocket_names:
        pocket = pocket_name.split('_')[-1]
        pocket_dir = data_dir + set_name + '/' + pocket_name + '/'
        file_paths = os.listdir(pocket_dir)
        input_rdkit = [file for file in file_paths if file.split('.')[-1] == 'rdkit']
        for input_complex in tqdm(input_rdkit):
            # 提取文件名（不包含扩展名）
            shutil.move(pocket_dir + input_complex, set_path + input_complex.replace('Complex', f'Complex_{pocket}'))